# "2019 June One Day Holding Long Short Portfolio BackTest"
> "2019 June Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [86]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [87]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')

In [88]:
#hide
df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190601].loc[df_testp.Date<20190701].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190601].loc[df_test.Date<20190701].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190601].loc[df_tes.Date<20190701].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [89]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [90]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a,cumreturn1b
0,2019-06-03,-1.0,-3.571429,1.035714,1.021432,1.020645,0.641543,0.947800
1,2019-06-03,-1.0,-3.571429,1.035714,1.021432,1.020645,0.997751,0.947800
2,2019-06-03,1.0,6.233486,1.100275,1.021432,1.020645,0.641543,0.947800
3,2019-06-03,1.0,6.233486,1.100275,1.021432,1.020645,0.997751,0.947800
4,2019-06-04,-1.0,2.397971,1.073891,1.029769,1.029001,1.097679,0.922666
...,...,...,...,...,...,...,...,...
59,2019-06-27,-1.0,11.111111,1.148706,1.071894,1.071724,0.026756,1.058304
60,2019-06-28,-1.0,-2.922222,1.182273,1.071894,1.071724,0.035975,1.420155
61,2019-06-28,-1.0,-2.922222,1.182273,1.071894,1.071724,0.040208,1.420155
62,2019-06-28,1.0,-13.032551,1.028193,1.071894,1.071724,0.035975,1.420155


In [92]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs 10QPortfolio (yellow) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
    x='Date:T',
    y='cumreturn1b:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5

alt.LayerChart(...)

**Long portfolio**

In [93]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [94]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [95]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a,cumreturn1b
0,2019-06-07,1.0,0.000000,1.064299,1.051843,1.050077,1.091047,1.006817
1,2019-06-11,1.0,-3.898957,1.022802,1.049332,1.047746,0.920325,1.014562
2,2019-06-12,1.0,-2.216749,1.000129,1.053632,1.051853,1.102184,1.001824


In [96]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs 10QPortfolio (yellow) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
    x='Date:T',
    y='cumreturn1b:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5

alt.LayerChart(...)

**Short Portfolio**

In [97]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [98]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [99]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a,cumreturn1b
0,2019-06-03,-1.0,-3.571429,1.035714,1.021432,1.020645,0.641543,0.947800
1,2019-06-04,-1.0,2.397971,1.010878,1.029769,1.029001,0.705796,0.922666
2,2019-06-05,-1.0,1.017551,1.000592,1.036087,1.036297,0.925371,0.930770
3,2019-06-07,-1.0,2.459741,0.975980,1.051843,1.050077,0.731157,0.776422
4,2019-06-10,-1.0,0.000000,0.975980,1.051475,1.049506,0.560910,0.766041
5,2019-06-11,-1.0,0.000000,0.975980,1.049332,1.047746,0.644335,0.817408
6,2019-06-13,-1.0,1.571483,0.960643,1.051934,1.051162,0.008465,0.901679
7,2019-06-14,-1.0,-12.256856,1.078387,1.052914,1.052085,0.009414,0.948783
8,2019-06-18,-1.0,8.929171,0.982096,1.066319,1.067858,0.007124,0.948783
9,2019-06-24,-1.0,8.972461,1.028916,1.063011,1.069640,0.025892,0.964795


In [100]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs 10QPortfolio (yellow) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
    x='Date:T',
    y='cumreturn1b:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5

alt.LayerChart(...)